<img src="img/1.png">

Estimador de momentos:

Uso el primer momento, pues si $b > 0$ con $\mathcal{U}[0,b]$ \, 

o con $b < 0$ para $\mathcal{U}[b,0]$, la esperanza será distinta de cero.

\begin{equation*}
\hat{b}_{mom} = E [X_i] = \int_{0}^{b} x_i * \frac{1}{b} * dx = latex es muerte
\end{equation*}












In [109]:
# Función estimadora de primeros momentos
bmom1 = function(muestra){
    2*mean(muestra)
}

# Función estimadora de segundos momentos (para comparar)
bmom2 = function(muestra){
    n <- length(muestra)
    sqrt(3 * mean(muestra^2))
}

In [110]:
# Estimador de maxima verosimilitud
bmv = function(muestra){
    max(muestra)
}

In [231]:
b <- 1
n <- 1500000
muestra <- runif(n, min=0, max=b)

#acumuladores
bmom1_acum <- 0
bmom2_acum <- 0
bmv_acum   <- 0
# error acumulado en k experimentos
k <- 1000
for(i in 1:k){
    bmom1_acum <- bmom1_acum + abs(bmom1(muestra)-b)
    bmom2_acum <- bmom2_acum + abs(bmom2(muestra)-b)
    bmv_acum   <- bmv_acum   + abs(bmv(muestra)-b)
}
bmom1_acum
bmom2_acum
bmv_acum 

[1] 0.006333609

[1] 0.005034935

[1] 0.0004733447

<img src="img/2.png">

In [112]:
bmed = function(muestra){
    2*median(muestra)
}

<img src="img/3.png">

In [113]:
b <- 1
n <- 15
muestra <- runif(n, min=0, max=b)

In [115]:
b - bmom1(muestra)
b - bmom2(muestra)
b - bmv(muestra)
b - bmed(muestra)

[1] 0.01767916

[1] 0.02165576

[1] 0.09393157

[1] -0.0725689

<img src="img/4.png">

In [221]:
experimento = function(){
    # a)
    b <- 1
    n <- 15
    muestra <- runif(n, min=0, max=b)
    # b)
    b_mom1 <- bmom1(muestra)
    b_mom2 <- bmom2(muestra)
    b_mv  <- bmv(muestra)
    b_med <- bmed(muestra)
    #devuelvo un vector de estimadores
    c(b_mom1, b_mom2, b_mv, b_med)
}

In [222]:
# c)
nrep <- 10
estimadores <- array(dim=c(nrep,4), dimnames=list(1:nrep, c("b_mom1", "b_mom2", "b_mv", "b_med")))
for(i in 1:nrep){
    estimadores[i,] <- array(experimento())
}

In [223]:
estimadores[1:3,]

b_mom1,b_mom2,b_mv,b_med
0.6579970,0.6776901,0.6116319,0.6756847
0.9005615,0.8977722,0.9781022,0.8605202
0.7909920,0.8367311,0.8973918,0.6484276


#### Media uniforme: E[X] = (a+b)/2

In [224]:
# d)
medias_muestrales <- estimadores/2 # + 0/2, pues a = 0, y b = estimadores

In [233]:
sesgos <- medias_muestrales - b

sesgo_aprox <- apply(sesgos, MARGIN=2, FUN=median)

In [234]:
sesgo_aprox

b_mom1     b_mom2       b_mv      b_med 
-0.5188278 -0.5199649 -0.5260741 -0.5546133

#### Varianza muestra: Uso estimador insesgado: n/(n-1) * S^2

In [235]:
# e)
varianzas_muestrales <- ((medias_muestrales-b/2)^2)/(n-1)

In [237]:
varianza_aprox <- apply(varianzas_muestrales, MARGIN=2, FUN=mean)

In [238]:
varianza_aprox

b_mom1       b_mom2         b_mv        b_med 
3.821832e-09 2.808353e-09 2.970262e-09 7.478889e-09

In [242]:
# f) Aproximación del Error Cuadratico Medio (ECM)
ECM <- varianza_aprox + sesgo_aprox^2
ECM

b_mom1    b_mom2      b_mv     b_med 
0.2691823 0.2703635 0.2767540 0.3075959

<img src="img/5.png">

In [ ]:
#simulacion_mv(b,n)
simulacion_mv = function(b, n){
    muestra <- runif(n, min=0, max=b)
    bmv(muestra)
}

simulacion_mom = function(b, n){
    muestra <- runif(n, min=0, max=b)
    bmom1(muestra)
}

In [ ]:
# Función estimadora de primeros momentos
bmom1 = function(muestra){
    2*mean(muestra)
}

# Función estimadora de segundos momentos (para comparar)
bmom2 = function(muestra){
    n <- length(muestra)
    sqrt(3 * mean(muestra^2))
}

# Estimador de maxima verosimilitud
bmv = function(muestra){
    max(muestra)
}